# braun

In [1]:
import warnings
import os
import sys
import gc
import warnings

In [2]:
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
tardis.config = tardis.config_server

In [3]:
print(f"CUDA used: {torch.cuda.is_available()}")

CUDA used: False


In [4]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [5]:
adata_file_path = os.path.join(tardis.config.io_directories["processed"], "dataset_complete_Braun.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path)
print(adata_file_path, flush=True)
print(adata, flush=True)

/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/processed/dataset_complete_Braun.h5ad
AnnData object with n_obs × n_vars = 1661498 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes'
    uns: 'rank_genes_groups'
    obsm: 'Unintegrated', 'X_pca', 'harmony'


In [6]:
with pd.option_context('display.max_rows', None):
    k = adata.obs.groupby(["organ", "integration_library_platform_coarse"]).size()
    display(k)

organ  integration_library_platform_coarse
Brain  3GEX                                   1661498
dtype: int64

In [7]:
with pd.option_context('display.max_rows', None):
    k = adata.obs.groupby(["sex", "age", "integration_donor"]).size()
    display(k[k!=0])

sex  age   integration_donor   
F    5.0   XDD:348_Braun_et_al      62751
     5.5   XDD:400_Braun_et_al      59667
     6.0   XDD:326_Braun_et_al      22198
     6.6   BRC2106A_Braun_et_al     61996
     6.9   BRC2061_Braun_et_al     128294
     7.0   XDD:398_Braun_et_al      17653
     7.5   XHU:305_Braun_et_al      38793
     8.0   XDD:334_Braun_et_al      65511
     8.1   BRC2057_Braun_et_al      31456
     8.5   XDD:313_Braun_et_al      50605
     9.2   XHU:307_Braun_et_al      85820
     9.5   XHU:292_Braun_et_al      28452
     10.0  XHU:297_Braun_et_al      88790
     11.5  XDD:358_Braun_et_al     102454
     13.0  XDD:359_Braun_et_al      38865
M    6.0   XDD:395_Braun_et_al      85990
     6.6   BRC2073_Braun_et_al      24790
     6.7   BRC2147_Braun_et_al      79580
     6.9   BRC2110_Braun_et_al      60345
           BRC2114_Braun_et_al      76439
           BRC2191_Braun_et_al     158541
     8.0   BRC2006_Braun_et_al      47021
           BRC2021_Braun_et_al      13718
  

In [8]:
adata = adata[
    (adata.obs["sex"] == "F")
]

In [9]:
with pd.option_context('display.max_rows', None):
    k = adata.obs.groupby(["age", "integration_donor"]).size()
    display(k[k!=0])

age   integration_donor   
5.0   XDD:348_Braun_et_al      62751
5.5   XDD:400_Braun_et_al      59667
6.0   XDD:326_Braun_et_al      22198
6.6   BRC2106A_Braun_et_al     61996
6.9   BRC2061_Braun_et_al     128294
7.0   XDD:398_Braun_et_al      17653
7.5   XHU:305_Braun_et_al      38793
8.0   XDD:334_Braun_et_al      65511
8.1   BRC2057_Braun_et_al      31456
8.5   XDD:313_Braun_et_al      50605
9.2   XHU:307_Braun_et_al      85820
9.5   XHU:292_Braun_et_al      28452
10.0  XHU:297_Braun_et_al      88790
11.5  XDD:358_Braun_et_al     102454
13.0  XDD:359_Braun_et_al      38865
dtype: int64

In [10]:
adata.shape

(883305, 8192)

In [11]:
with pd.option_context('display.max_rows', None):
    k = adata.obs.groupby(["age", "sex", "integration_donor", "integration_library_platform_coarse", "organ", "concatenated_integration_covariates"]).size()
    display(k[k!=0])

age   sex  integration_donor     integration_library_platform_coarse  organ  concatenated_integration_covariates 
5.0   F    XDD:348_Braun_et_al   3GEX                                 Brain  XDD:348_Braun_et_al_Cell_Fresh_3GEX      62751
5.5   F    XDD:400_Braun_et_al   3GEX                                 Brain  XDD:400_Braun_et_al_Cell_Fresh_3GEX      59667
6.0   F    XDD:326_Braun_et_al   3GEX                                 Brain  XDD:326_Braun_et_al_Cell_Fresh_3GEX      22198
6.6   F    BRC2106A_Braun_et_al  3GEX                                 Brain  BRC2106A_Braun_et_al_Cell_Fresh_3GEX     61996
6.9   F    BRC2061_Braun_et_al   3GEX                                 Brain  BRC2061_Braun_et_al_Cell_Fresh_3GEX     128294
7.0   F    XDD:398_Braun_et_al   3GEX                                 Brain  XDD:398_Braun_et_al_Cell_Fresh_3GEX      17653
7.5   F    XHU:305_Braun_et_al   3GEX                                 Brain  XHU:305_Braun_et_al_Cell_Fresh_3GEX      38793
8.0   F    XDD:334

In [12]:
df = adata.obs.copy()
group_sizes = df.groupby(["cell_type", "age"]).size()
valid_groups = group_sizes[group_sizes > 30].index
filtered_bool = df.set_index(["cell_type", "age"]).index.isin(valid_groups)
print(filtered_bool.sum())
adata = adata[filtered_bool]

882450


In [13]:
with pd.option_context('display.max_rows', None):
    k = adata.obs.groupby(["cell_type", "age"]).size()
    display(k[k>0])

cell_type                                   age 
Brain erythrocytes                          5.0       219
                                            5.5       464
                                            6.0       113
                                            6.6       111
                                            6.9       112
                                            7.0        61
                                            7.5        40
                                            8.5       275
                                            9.2        74
                                            9.5        36
                                            10.0       37
                                            11.5     1047
                                            13.0      149
Brain fibroblasts                           5.0      6093
                                            5.5      7165
                                            6.6       406
                       

In [14]:
adata

View of AnnData object with n_obs × n_vars = 882450 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes'
    uns: 'rank_genes_groups'
    obsm: 'Unintegrated', 'X_pca', 'harmony'

In [15]:
adata_write_path = os.path.join(tardis.config.io_directories["processed"], "dataset_complete_Braun_age.h5ad")
adata.write_h5ad(adata_write_path)